###  Grabs face ROIs from input live video or files and to create a deep learning face liveness dataset.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import os

In [3]:
args = {
    'input':'./videos/real.mov', # path to input video
    'output': './dataset/fake', # path to output directory of cropped faces
    'detector': './face_detector/', # path to OpenCV's deep learning face detector
    'confidence': 0.5, # minimum probability to filter weak detections
    'skip': 1, # # of frames to skip before applying face detection
    
}

In [4]:
# load our serialized face detector from disk
print("[INFO] loading face detector")
print('...')
protoPath = os.path.sep.join([args["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([args["detector"], "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("[INFO] face detector loaded")

[INFO] loading face detector
...
[INFO] face detector loaded


In [5]:
# open a pointer to the video file stream and initialize the total
# number of frames read and saved thus far
# for live camera set VideoCapture to '0'
vs = cv2.VideoCapture(args["input"])
read = 0
saved = 0
counter = 0

In [6]:
# loop over frames from the video file stream
while True:
    # grab the frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # increment the total number of frames read thus far
    read += 1

    # check to see if we should process this frame
    if read % args["skip"] != 0:
        continue

    # grab the frame dimensions and construct a blob from the frame
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()

    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > args["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the face and extract the face ROI
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face = frame[startY:endY, startX:endX]

            # write the frame to disk
            p = os.path.sep.join([args["output"],
                "{}.png".format(saved)])
            cv2.imwrite(p, face)
            saved += 1
            if counter < 5:
                print("[INFO] saved {} to disk".format(p))
                counter +=1


[INFO] saved ./dataset/fake/0.png to disk
[INFO] saved ./dataset/fake/1.png to disk
[INFO] saved ./dataset/fake/2.png to disk
[INFO] saved ./dataset/fake/3.png to disk
[INFO] saved ./dataset/fake/4.png to disk


In [7]:
# do a bit of cleanup
vs.release()
cv2.destroyAllWindows()